In [48]:
## Convert mapping from xlsx to CSV
import pandas as pd

files = [
    ("Data/BAMS_database/bams2_connectivity.xlsx", "All regions connected", "Data/csvs/bams2_connectivity/bams2_mapping.csv", {}),
]

for xlsx, sheet, csv, converters in files:
    data_xls = pd.read_excel(xlsx, sheet, index_col=None, converters = converters)
    data_xls.to_csv(csv, encoding='utf-8')
    
print("Converted all regions from xlsx to csv")

Converted all regions from xlsx to csv


In [1]:
from neo4j import GraphDatabase, basic_auth
from dotenv import load_dotenv
import os

load_dotenv()

neo4jUser = os.getenv("NEO4J_USER")
neo4jPwd = os.getenv("NEO4J_PASSWORD")

driver = GraphDatabase.driver("bolt://localhost:7687",auth=basic_auth(neo4jUser, neo4jPwd))

In [11]:
def get_count_of_type(label, session):
    q = "MATCH (n:%s) RETURN count(n)" % label
    res = session.run(q)
    print("Added", res.value()[0], "nodes of type", label)
    
def get_count_of_relationship(label, session):
    q = "MATCH ()-[r:%s]-() RETURN count(*)" %label
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", label)

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/bams2_connectivity")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [50]:
import pandas as pd

## Load csv
csv_file_path = "file:///%s" % get_csv_path("bams2_mapping.csv")
df = pd.read_csv(csv_file_path)

class Region:
    
    def __init__(self, reg_id, name, abbr):
        self.id = reg_id
        self.name = name
        self.abbreviation = abbr
        self.relates_to = []
        
    def setRelatesTo(self, region_ids):
        self.relates_to = region_ids
        
    
    
bams_regions = {}
relation_types = {"part of": 1, "overlapping": 2, "identical": 3, "includes": 4}

for index, row in df.iterrows():
    if(pd.isnull(row["All sorted"])):
        continue
    name_split =  row["All sorted"].split("(")
    name = name_split[0].strip()
    abbreviation = name_split[1][:-1]
    reg_id = index+1
    reg = Region(reg_id, name, abbreviation)
    
    # extract BG region ids
    rel_str = row["Relates to"]
    
    bg_ids = []
    rel_str_splitted = rel_str.split()
    for j in range(len(rel_str_splitted)):
        digit = rel_str_splitted[j]
        if (digit.isdigit() and j < len(rel_str_splitted)-1 and rel_str_splitted[j+1].startswith("WHS")):
            bg_ids.append(digit)
    
    rel_types = row["Relation type"].split(", ")
    
    ## Add bg reltation to Bams region class
    reg_relations = []
    for i in range(len(bg_ids)):
        reg_relations.append((bg_ids[i], rel_types[i]))
    
    reg.setRelatesTo(reg_relations)
    
    bams_regions[reg_id] = reg




In [ ]:
## TODO NEXT: Add the new regions to BG region

## THEN add the relationships